In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
device=('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
df = pd.read_csv('data/sonar3.csv', header=None)
X=df.iloc[:, :-1]
y=df.iloc[:, -1]
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, shuffle=True)

In [4]:
def df_to_tensor(df):
    return torch.from_numpy(df.values).float().to(device)

In [5]:
X_train=df_to_tensor(X_train)
X_test=df_to_tensor(X_test)
y_train=df_to_tensor(y_train)
y_test=df_to_tensor(y_test)

In [6]:
class Mineral(nn.Module):
    def __init__(self):
        super(Mineral, self).__init__()
        self.hidden_linear=nn.Linear(60, 24)
        self.hidden_activation=nn.ReLU()
        self.hidden_linear2=nn.Linear(24, 10)
        self.hidden_activation2=nn.ReLU()
        self.output_linear=nn.Linear(10,1)
        self.output_activation=nn.Sigmoid()
        
    def forward(self, x):
        x=self.hidden_linear(x)
        x=self.hidden_activation(x)
        x=self.hidden_linear2(x)
        x=self.hidden_activation2(x)
        x=self.output_linear(x)
        x=self.output_activation(x)
        
        return x

In [7]:
model=Mineral().to(device)
model

Mineral(
  (hidden_linear): Linear(in_features=60, out_features=24, bias=True)
  (hidden_activation): ReLU()
  (hidden_linear2): Linear(in_features=24, out_features=10, bias=True)
  (hidden_activation2): ReLU()
  (output_linear): Linear(in_features=10, out_features=1, bias=True)
  (output_activation): Sigmoid()
)

In [8]:
def count_parameters(model):
    total_param = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_param = np.prod(param.size())
            if param.dim() > 1:
                print(name, ':', ' x '.join(str(x) for x in list(param.size())[::-1]), '=', num_param)
            else:
                print(name, ':', num_param)
                print('-' * 40)
            total_param += num_param
    print('total:', total_param)

count_parameters(model)

hidden_linear.weight : 60 x 24 = 1440
hidden_linear.bias : 24
----------------------------------------
hidden_linear2.weight : 24 x 10 = 240
hidden_linear2.bias : 10
----------------------------------------
output_linear.weight : 10 x 1 = 10
output_linear.bias : 1
----------------------------------------
total: 1725


In [9]:
batch_size=10
ds=TensorDataset(X_train, y_train)
dataloader= DataLoader(ds, batch_size=batch_size)
optimizer=optim.Adam(model.parameters())

n_epochs=100
loss_fn=nn.BCELoss()

for epoch in range(n_epochs):
    for data, label in dataloader:
        out=model(data.to(device))
        loss=loss_fn(out, label.unsqueeze(1).to(device)) 
        loss.backward() 
        optimizer.step()
        optimizer.zero_grad() 
    print("Epoch: %d, Loss: %.4f" %(epoch, float(loss)))
    
torch.save(model.state_dict(), 'model/mineral.pth') #layer정보는 없고 학습한 파라미터만 뽑아냄

Epoch: 0, Loss: 0.6852
Epoch: 1, Loss: 0.6857
Epoch: 2, Loss: 0.6745
Epoch: 3, Loss: 0.6660
Epoch: 4, Loss: 0.6549
Epoch: 5, Loss: 0.6382
Epoch: 6, Loss: 0.6158
Epoch: 7, Loss: 0.5775
Epoch: 8, Loss: 0.5431
Epoch: 9, Loss: 0.5030
Epoch: 10, Loss: 0.4577
Epoch: 11, Loss: 0.4136
Epoch: 12, Loss: 0.3744
Epoch: 13, Loss: 0.3380
Epoch: 14, Loss: 0.3065
Epoch: 15, Loss: 0.2799
Epoch: 16, Loss: 0.2572
Epoch: 17, Loss: 0.2405
Epoch: 18, Loss: 0.2242
Epoch: 19, Loss: 0.2127
Epoch: 20, Loss: 0.2022
Epoch: 21, Loss: 0.1935
Epoch: 22, Loss: 0.1851
Epoch: 23, Loss: 0.1794
Epoch: 24, Loss: 0.1725
Epoch: 25, Loss: 0.1667
Epoch: 26, Loss: 0.1615
Epoch: 27, Loss: 0.1566
Epoch: 28, Loss: 0.1521
Epoch: 29, Loss: 0.1483
Epoch: 30, Loss: 0.1432
Epoch: 31, Loss: 0.1389
Epoch: 32, Loss: 0.1337
Epoch: 33, Loss: 0.1302
Epoch: 34, Loss: 0.1248
Epoch: 35, Loss: 0.1211
Epoch: 36, Loss: 0.1172
Epoch: 37, Loss: 0.1146
Epoch: 38, Loss: 0.1099
Epoch: 39, Loss: 0.1087
Epoch: 40, Loss: 0.1048
Epoch: 41, Loss: 0.1012
Ep

In [10]:
model.load_state_dict(torch.load('model/mineral.pth', map_location=device))
test_ds=TensorDataset(X_test, y_test)
test_loader=DataLoader(test_ds, batch_size=batch_size)

correct=0

with torch.no_grad():
    for data, label in test_loader:
        pred=model(data.to(device))
        result=pred.squeeze(1).ge(torch.tensor(0.5).to(device)) #1차원으로 변경, greater than equal, batch size 만큼의 TF
        correct+=result.long().eq(label.to(device)).sum().item() #T=1, F=0
        
print("Accuracy: %.4f" % (correct / len(test_loader.dataset)))

Accuracy: 0.7143
